In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
!make -C csrc/bcc/libbpf-tools

make: Entering directory '/Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools'
make: Nothing to be done for 'all'.
make: Leaving directory '/Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools'


In [3]:
!make -C csrc/bpftool/src bpftool

make: Entering directory '/Users/szhong/Downloads/bpf-study/csrc/bpftool/src'
...                        libbfd: [ OFF ]
...               clang-bpf-co-re: [ on  ]
...                          llvm: [ on  ]
...                        libcap: [ OFF ]
make: 'bpftool' is up to date.
make: Leaving directory '/Users/szhong/Downloads/bpf-study/csrc/bpftool/src'


In [4]:
import pandas as pd
from typing import List
from pathlib import Path
from depsurf import OUTPUT_PATH, DepKind, Dep, diff_dep, Version, VERSIONS_REGULAR, BCC_OBJ_PATH, BPFObject
from depsurf.output import save_df
import logging

class Report:
    def __init__(self, name, deps: List[Dep], versions: list[Version], path: Path):
        self.name = name
        self.deps = deps
        self.versions = versions
        self.path = path
        self.file = open(path, "w")

    def __del__(self):
        self.file.close()

    @staticmethod
    def deps_filter(dep: Dep) -> bool:
        if dep.kind in (DepKind.STRUCT_FIELD, DepKind.STRUCT):
            return not dep.name.startswith("user_pt_regs")

        if dep.kind == DepKind.FUNC:
            return not (dep.name.startswith("dummy") or dep.name == "foo")

        if dep.kind in (DepKind.TRACEPOINT, DepKind.LSM):
            return True

        return False

    def report(self):
        import pandas as pd

        result_version = {}
        result_delta = {}

        for dep in self.deps:
            if not Report.deps_filter(dep):
                continue
            print(f"{dep.kind:12}{dep.name}", file=self.file)
            key = (self.name, dep.kind, dep.name)
            result_version[key], result_delta[key] = self.report_dep(dep)

        logging.info(f"Report saved to {self.path}")

        df_version = pd.DataFrame(result_version).T
        df_delta = pd.DataFrame(result_delta).T

        return df_version, df_delta

    def report_dep(self, dep: Dep):
        result_version = {}
        result_delta = {}

        for v in self.versions:
            result_version[v.short_version] = v.img.get_dep_status(dep)

        version_str = "|".join(map(str, result_version.values()))
        print(f"\tVersion Status: {version_str}", file=self.file)

        for v1, v2 in zip(self.versions, self.versions[1:]):
            changes = diff_dep(v1.img, v2.img, dep)
            if changes is None:
                num = -1
            elif not changes:
                num = 0
            else:
                num = len(changes)
                print(f"\t{v1.short_version} -> {v2.short_version}: {num} changes", file=self.file)
                for change in changes:
                    print(f"\t\t{change}", file=self.file)
            result_delta[(v1.short_version, v2.short_version)] = num

        return result_version, result_delta

result_version = []
result_delta = []
for path in sorted(BCC_OBJ_PATH.glob("*.bpf.o")):
    bpf = BPFObject(path)
    result_path = OUTPUT_PATH / "bcc" / f"{bpf.name}.log"
    report = Report(bpf.name, bpf.deps, VERSIONS_REGULAR, result_path)
    cs, cd = report.report()
    result_version.append(cs)
    result_delta.append(cd)


df_delta = pd.concat(result_delta)
df_delta.index.names = ["Program", "Type", "Name"]
save_df(df_delta, "bcc_delta")


df_version = pd.concat(result_version)
df_version.index.names = ["Program", "Type", "Name"]
save_df(df_version, "bcc_version")

print("Done")

[    decorator.py:17 ] INFO: gen_min_btf      Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bashreadline.bpf.min.btf
[    decorator.py:17 ] INFO: dump_btf_impl    Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bashreadline.bpf.min.btf.json
[    decorator.py:17 ] INFO: dump_btf_impl    Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bashreadline.bpf.min.btf.txt
[   1495636915.py:45 ] INFO: Report saved to /Users/szhong/Downloads/bpf-study/output/bcc/bashreadline.log
[    decorator.py:17 ] INFO: gen_min_btf      Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bindsnoop.bpf.min.btf
[    decorator.py:17 ] INFO: dump_btf_impl    Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bindsnoop.bpf.min.btf.json
[    decorator.py:17 ] INFO: dump_btf_impl    Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bindsnoop.bpf.min.btf.txt
[  func_groups.py:68 ] INFO: Loading f

Done


In [5]:
from depsurf.output import load_df

df_delta = load_df("bcc_delta")
df_delta = df_delta.replace({0: ".", -1: ""})
df_delta


[           pd.py:50 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/bcc_delta.pkl


4.4  4.8 4.10 4.13 4.15 4.18 5.0 5.3 5.4  5.8 5.11 5.13 5.15 5.19 6.2 6.5
                                                                        4.8 4.10 4.13 4.15 4.18  5.0 5.3 5.4 5.8 5.11 5.13 5.15 5.19  6.2 6.5 6.8
Program     Type       Name                                                                                                                      
bindsnoop   Function   inet6_bind                                         .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       inet_bind                                          .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
            Field      in6_addr::in6_u                                    .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       inet_sock::bind_address_no_port                    .    .    .    .    .    .   .   .   .    .    .    .    .    .   .    
                       inet_sock::freebind                                .    .    .    .    .    .   .   .   .    .    .    .    .    .   .    
                       inet_sock::inet_flags                                                                                                     
                       inet_sock::inet_saddr                              .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       inet_sock::inet_sport                              .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       inet_sock::transparent                             .    .    .    .    .    .   .   .   .    .    .    .    .    .   .    
                       sock::__sk_common                                  .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       sock::sk_protocol                                  .    .    .    .    .    .   .   .   1    .    .    .    .    .   .   .
                       sock_common::skc_bound_dev_if                      .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       sock_common::skc_reuse                             .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       sock_common::skc_reuseport                         .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       sock_common::skc_v6_rcv_saddr                      .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       socket::sk                                         .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
            Struct     in6_addr                                           .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       inet_sock                                          .    .    1    .    .    .   .   .   .    1    .    1    .    .   2  14
                       sock                                               9    3    4    3    2    6   3   .   8    3    .    5    9    3   1   3
                       sock_common                                        1    .    1    .    .    2   .   .   .    .    .    .    .    .   .   .
                       socket                                             .    .    .    .    .    .   1   .   .    .    .    .    .    .   .   .
biolatency  Tracepoint block_rq_complete                                  .    .    5    .    .    .   .   .   .    .    .    .    1    .   .   .
                       block_rq_insert                                    .    .    .    .    .    .   .   .   .    2    .    .    .    .   .   .
                       block_rq_issue                                     .    .    .    .    .    .   .   .   .    2    .    .    .    .   .   .
            Field      gendisk::first_minor                               .    .    .    .    .    .   .   .   .    .    .    .   

In [6]:
from depsurf.output import load_df

df_version = load_df("bcc_version")

df_version.loc[(slice(None), 'Function', slice(None)), :]

[           pd.py:50 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/bcc_version.pkl


4.4                  4.8                 4.10                 4.13                 4.15                 4.18                  5.0                  5.3                  5.4                          5.8                         5.11                         5.13                         5.15                         5.19                          6.2                          6.5                          6.8
Program     Type     Name                                                                                                                                                                                                                                                                                                                                                                                                                                                         
bindsnoop   Function inet6_bind                                  ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                            ✅                            ✅                            ✅                            ✅                            ✅                            ✅                            ✅                            ✅
                     inet_bind                                   ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                            ✅                            ✅                            ✅                            ✅                            ✅                            ✅                            ✅                            ✅
biosnoop    Function blk_account_io_merge_bio                    ❌                    ❌                    ❌                    ❌                    ❌                    ❌                    ❌                    ❌                    ❌  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined
                     blk_account_io_start                        ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                            ✅                            ✅          🟡 Partially inlined          🟡 Partially inlined      🟣 Static🟠 Fully inlined      🟣 Static🟠 Fully inlined      🟣 Static🟠 Fully inlined      🟣 Static🟠 Fully inlined
biostacks   Function blk_account_io_done       🟡 Partially inlined  🟡 Partially inlined  🟡 Partially inlined  🟡 Partially inlined  🟡 Partially inlined  🟡 Partially inlined                    ✅                    ✅                    ✅                            ✅                            ✅                            ✅                            ✅      🟣 Static🟠 Fully inlined      🟣 Static🟠 Fully inlined                     🟣 Static                     🟣 Static
                     blk_account_io_merge_bio                    ❌                    ❌                    ❌                    ❌                    ❌                    ❌                    ❌                    ❌                    ❌  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined  🟣 Static🟡 Partially inlined
                     blk_account_io_start                        ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                    ✅                            ✅                            ✅          🟡 Partially inlined      